# Milestone 2

## Introduction

## Dataset overview

### Available fields

name, origin, packinging, composition

### Data completeness

Sparse dataset, lots of quasi empty columns

### General cleaning

What to do with empty columns

### Reliability of the dataset

crowd-sourced, blabla

## Question 1: How do nutrients influence the selected nutrition scores?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we keep the question, further investigations

## Question 2: Comparing bio vs. normal products, is there a real difference in composition?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we keep the question, further investigations

## Question 3: What are the levels of interdependencies between nations based on their production and importations?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we **don't keep** the question, further investigations

## Conclusion: proposed data story